<a href="https://colab.research.google.com/github/iliruslanili/erahackaton/blob/master/OrientedBoundingBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id="1EZmppVaQLkEJ5yMPbUIVFyucC7HxPyA0", dest_path="/content/ethalon.zip", showsize="True", )
gdd.download_file_from_google_drive(file_id="1mqOjNktZNIBZuc2QwS7jTRguQfKWzDUj", dest_path="/content/point_cloud_train.zip", showsize="True", )
gdd.download_file_from_google_drive(file_id="10LUo7rwt7tMUMtp1_t--bt4QazpwI_Dy", dest_path="/content/point_cloud_gt.zip", showsize="True", )

101.8 MiB Done.


In [3]:
!unzip ethalon.zip
!unzip ethalon/cloud_0_0003.pcd.zip
!unzip ethalon/cloud_2_0003.pcd.zip

Archive:  ethalon.zip
   creating: эталонный_пример_для_высылания_результата/
  inflating: эталонный_пример_для_высылания_результата/cloud_0_0003.pcd.json  
 extracting: эталонный_пример_для_высылания_результата/cloud_0_0003.pcd.zip  
  inflating: эталонный_пример_для_высылания_результата/cloud_2_0003.pcd.zip  
  inflating: эталонный_пример_для_высылания_результата/img_spvp_c1_t0_raw_0003.png  
 extracting: эталонный_пример_для_высылания_результата/readme.txt  
Archive:  ethalon/cloud_0_0003.pcd.zip
  inflating: cloud_0_0003.pcd        
Archive:  ethalon/cloud_2_0003.pcd.zip
  inflating: cloud_2_0003.pcd        


In [ ]:
!unzip point_cloud_train.zip

In [ ]:
!unzip point_cloud_gt.zip

In [ ]:
!mkdir -p pcd/clouds_tof
!unzip "point_cloud_train/clouds_tof/*.zip" -d pcd/clouds_tof

In [ ]:
!mkdir -p pcd/clouds_stereo
!unzip "point_cloud_train/clouds_stereo/*.zip" -d pcd/clouds_stereo

In [ ]:
!mkdir -p pcd_train/clouds_tof
!cp -r point_cloud_gt/clouds_tof_ann pcd_train
!unzip "point_cloud_gt/clouds_tof/*.zip" -d pcd_train/clouds_tof

In [ ]:
!mkdir -p pcd_train/clouds_stereo
!cp -r point_cloud_gt/clouds_stereo_ann pcd_train
!unzip "point_cloud_gt/clouds_stereo/*.zip" -d pcd_train/clouds_stereo

In [ ]:
!pip install open3d
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

In [27]:
def annot_upd(ann):
  dict1 = {'6bc1d79329bd444c8fb0717e408dd48c': 'human', 
           'f8b0fcbf2fd140caa49049729ad18072': 'wear',
           'dc58658c7ffd4802a1dd62258b4ad985': 'other',
           'e7eb42dd5e0544eb9f667a3915a64700': 'limb'}
  for i, figure in enumerate(ann['figures']):
    if figure['objectKey'] == '6bc1d79329bd444c8fb0717e408dd48c':
      ann['figures'][i]['object'] = 'human'
    if figure['objectKey'] == 'f8b0fcbf2fd140caa49049729ad18072':
      ann['figures'][i]['object'] = 'wear'
    if figure['objectKey'] == 'dc58658c7ffd4802a1dd62258b4ad985':
      ann['figures'][i]['object'] = 'other'
    if figure['objectKey'] == 'e7eb42dd5e0544eb9f667a3915a64700':
      ann['figures'][i]['object'] = 'limb'
  return ann

In [39]:
def createMeshFromBB(bb):
  bbcloud = o3d.geometry.PointCloud()
  bbcloud.points = bb.get_box_points()
  bb.color = bb.color
  hull, _= bbcloud.compute_convex_hull()
  return hull

In [38]:
def createMeshData(mesh, opacity=0.2):
  verts = np.asarray(mesh.vertices)
  triangs = np.asarray(mesh.triangles)

  mdata = go.Mesh3d(
        x=verts[:,0],
        y=verts[:,1],
        z=verts[:,2],
        i = triangs[:,0],
        j = triangs[:,1],
        k = triangs[:,2],
        opacity=opacity,
    )
  return mdata

In [37]:
def createScatter3dData(pcd):
  points = np.asarray(pcd.points)
  colors = np.asarray(pcd.colors)
  scdata = go.Scatter3d(
            x=points[:,0], 
            y=points[:,1], 
            z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors))
  return scdata

In [36]:
def drawGeometry(geometry):
  mydata = []
  for g in geometry:
    if type(g) is o3d.geometry.PointCloud:
      mydata.append(createScatter3dData(g))
    elif type(g) is o3d.geometry.AxisAlignedBoundingBox or type(g) is o3d.geometry.OrientedBoundingBox:
      m = createMeshFromBB(g)
      mydata.append(createMeshData(m))
    elif type(g) is o3d.geometry.TriangleMesh:
      mydata.append(createMeshData(g))
  
  fig = go.Figure(
    data=mydata,
    layout=dict(
        scene=dict(
            xaxis=dict(visible=True),
            yaxis=dict(visible=True),
            zaxis=dict(visible=True)
        )
    )
  )
  fig.show()

In [35]:
def showPointCloud(pcd):
  points = np.asarray(pcd.points)
  colors = np.asarray(pcd.colors)
  fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors)
        )
    ],
    layout=dict(width=600, height=600,
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
  )
  fig.show()

In [34]:
# Checks if a matrix is a valid rotation matrix.
import math
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :

    assert(isRotationMatrix(R))

    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])

    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])

In [72]:
def obb2json(obbs):
  json_data = {'figures': []}
  for obj, obb in enumerate(obbs):
    pos_x, pos_y, pos_z = obb.get_center()
    rot_x, rot_y, rot_z = rotationMatrixToEulerAngles(obb.R)
    dim_x, dim_y, dim_z = obb.extent
    pos = {'x': pos_x, 'y': pos_y, 'z': pos_z}
    rot = {'x': rot_x, 'y': rot_y, 'z': rot_z}
    dim = {'x': dim_x, 'y': dim_y, 'z': dim_z}
    geometry = {'position': pos, 'rotation': rot, 'dimensions': dim}
    json_data['figures'].append({'object': str(obj), 'geometry': geometry})
  return json_data

In [83]:
def get_obb(ann):
  x = ann['geometry']['position']['x']
  y = ann['geometry']['position']['y']
  z = ann['geometry']['position']['z']

  rot_x = ann['geometry']['rotation']['x']
  rot_y = ann['geometry']['rotation']['y']
  rot_z = ann['geometry']['rotation']['z']

  dim_x = ann['geometry']['dimensions']['x']
  dim_y = ann['geometry']['dimensions']['y']
  dim_z = ann['geometry']['dimensions']['z']

  rot_matrix = o3d.geometry.get_rotation_matrix_from_xyz((rot_x + np.pi, rot_y, rot_z))
  center = np.array([[x], [y], [z]], dtype=np.float64)
  extend = np.array([[dim_x], [dim_y], [dim_z]], dtype=np.float64)

  obb = o3d.geometry.OrientedBoundingBox(center, rot_matrix, extend)
  return obb

In [93]:
import json
import os

path = 'pcd_train/clouds_tof_ann/'
files = os.listdir(path)

with open(path + files[4], 'r') as f:
  annotation = json.load(f)
obbs = []
for figure in annotation['figures']:
  obbs.append(get_obb(figure))
print(obbs)

[OrientedBoundingBox: center: (0.349912, -0.505645, 3.12494), extent: 0.372763, 0.2947981.09266), OrientedBoundingBox: center: (-0.344695, 0.469197, 4.00896), extent: 0.674993, 0.550152.8874)]


In [96]:
path = 'pcd_train/clouds_tof_ann/'
files = os.listdir(path)

with open(path + files[6], 'r') as f:
  annotation = json.load(f)
obbs = []
for figure in annotation['figures']:
  obbs.append(get_obb(figure))
pcd = o3d.io.read_point_cloud('pcd_train/clouds_tof/' + files[6][:-5])
rot = o3d.geometry.get_rotation_matrix_from_xyz((np.pi, 0, 0))
downpcd = pcd.voxel_down_sample(voxel_size=0.02)
# downpcd.rotate(rot)
obbs.append(downpcd)
# indices = obb.get_point_indices_within_bounding_box(downpcd.points)
# obj_cloud = downpcd.select_by_index(indices)
# obj_cloud.paint_uniform_color([1,0,0])
drawGeometry(obbs)